# Pareto Efficiency Analysis for Topic Modeling

This notebook performs Pareto efficiency analysis on topic modeling results to identify optimal model configurations balancing coherence and topic diversity.

## Overview
- **Input**: Model evaluation results from OCTIS optimization
- **Analysis**: Pareto efficiency identification with two weighting strategies
- **Output**: Top models, visualizations, and hyperparameter analysis


## 1. Setup and Imports

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from scipy.stats import pearsonr, spearmanr
import math

# Set style for plots
sns.set_style("whitegrid")
plt.rcParams['figure.dpi'] = 100

# Define paths - calculate project root from current working directory
# Notebook is in notebooks/05_selection/, project root is 2 levels up
current_dir = Path.cwd()
if 'notebooks' in str(current_dir):
    # Running from notebook directory
    project_root = current_dir.parent.parent if '05_selection' in str(current_dir) else current_dir.parent
else:
    # Running from project root or elsewhere - try to find project root
    project_root = current_dir
    # Look for results directory to confirm we're in the right place
    if not (project_root / "results").exists():
        # Try going up one level
        project_root = current_dir.parent
        if not (project_root / "results").exists():
            # Last resort: assume we're in project root
            project_root = current_dir

input_csv = project_root / "results/experiments/bill_novels_octis_model_results_all_models.csv"
output_base = project_root / "results/pareto"

# Create output directories
(output_base / "figures").mkdir(parents=True, exist_ok=True)
(output_base / "tables").mkdir(parents=True, exist_ok=True)
(output_base / "top_models").mkdir(parents=True, exist_ok=True)

print(f"Current directory: {current_dir}")
print(f"Project root: {project_root}")
print(f"Input file: {input_csv}")
print(f"Input file exists: {input_csv.exists()}")
print(f"Output directory: {output_base}")


## 2. Data Loading


In [ ]:
# Load the model evaluation results
df = pd.read_csv(input_csv)

print(f"Loaded {len(df)} model evaluation results")
print(f"\nColumns: {list(df.columns)}")
print(f"\nShape: {df.shape}")
print(f"\nFirst few rows:")
df.head()


## 3. Data Cleaning

Remove outliers using the same approach as the original analysis:
1. Remove runs where Topic_Diversity or Coherence equals 1.0 (failed runs)
2. Remove outliers beyond 2 standard deviations from the mean (both upper and lower bounds)


In [ ]:
# Remove runs where Topic Diversity or Coherence is exactly 1 (failed runs)
df_clean = df[(df['Topic_Diversity'] < 1.0) & (df['Coherence'] < 1.0)].copy()
print(f"After removing failed runs (== 1.0): {len(df_clean)} models")

# Compute mean and standard deviation for outlier detection
coherence_mean = df_clean['Coherence'].mean()
coherence_std = df_clean['Coherence'].std()
topic_diversity_mean = df_clean['Topic_Diversity'].mean()
topic_diversity_std = df_clean['Topic_Diversity'].std()

# Define bounds (2 standard deviations)
coherence_lower = coherence_mean - 2 * coherence_std
coherence_upper = coherence_mean + 2 * coherence_std
topic_diversity_lower = topic_diversity_mean - 2 * topic_diversity_std
topic_diversity_upper = topic_diversity_mean + 2 * topic_diversity_std

print(f"\nCoherence bounds: [{coherence_lower:.4f}, {coherence_upper:.4f}]")
print(f"Topic Diversity bounds: [{topic_diversity_lower:.4f}, {topic_diversity_upper:.4f}]")

# Plot distributions with cutoff points
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Coherence distribution
axes[0].hist(df_clean['Coherence'], bins=20, edgecolor='black', alpha=0.7)
axes[0].axvline(coherence_lower, color='red', linestyle='dashed', linewidth=2, label='Lower Cutoff')
axes[0].axvline(coherence_upper, color='green', linestyle='dashed', linewidth=2, label='Upper Cutoff')
axes[0].set_title('Coherence Distribution with Cutoff Points')
axes[0].set_xlabel('Coherence')
axes[0].set_ylabel('Frequency')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Topic Diversity distribution
axes[1].hist(df_clean['Topic_Diversity'], bins=20, edgecolor='black', alpha=0.7)
axes[1].axvline(topic_diversity_lower, color='red', linestyle='dashed', linewidth=2, label='Lower Cutoff')
axes[1].axvline(topic_diversity_upper, color='green', linestyle='dashed', linewidth=2, label='Upper Cutoff')
axes[1].set_title('Topic Diversity Distribution with Cutoff Points')
axes[1].set_xlabel('Topic Diversity')
axes[1].set_ylabel('Frequency')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(output_base / "figures/distribution_with_cutoffs.png", dpi=150, bbox_inches='tight')
plt.show()

# Remove outliers
df_clean = df_clean[
    (df_clean['Coherence'] >= coherence_lower) & 
    (df_clean['Coherence'] <= coherence_upper) &
    (df_clean['Topic_Diversity'] >= topic_diversity_lower) & 
    (df_clean['Topic_Diversity'] <= topic_diversity_upper)
].reset_index(drop=True)

print(f"\nAfter removing outliers (2 std dev): {len(df_clean)} models")
print("\nDescriptive statistics after cleaning:")
df_clean[['Coherence', 'Topic_Diversity']].describe()


## 4. Metric Normalization

Apply Z-score normalization to make Coherence and Topic Diversity comparable for the combined score calculation.


In [ ]:
# Apply Z-score normalization
scaler = StandardScaler()
df_clean[['Coherence_norm', 'Topic_Diversity_norm']] = scaler.fit_transform(
    df_clean[['Coherence', 'Topic_Diversity']]
)

print("Normalized metrics (sample):")
df_clean[['Coherence', 'Coherence_norm', 'Topic_Diversity', 'Topic_Diversity_norm']].head()


## 5. Combined Score Calculation (Equal Weights)

Calculate combined score with equal weights for coherence and topic diversity (0.5 each).


In [ ]:
# Define weights for equal weighting strategy
weight_coherence = 0.5
weight_topic_diversity = 0.5

# Calculate combined score
df_clean['Combined_Score'] = (
    weight_coherence * df_clean['Coherence_norm'] + 
    weight_topic_diversity * df_clean['Topic_Diversity_norm']
)

# Rank by combined score
df_sorted = df_clean.sort_values(by='Combined_Score', ascending=False).reset_index(drop=True)

print(f"Top 10 Models Based on Combined Score (Equal Weights):")
top_10_equal = df_sorted.head(10)
display(top_10_equal[['Embeddings_Model', 'Iteration', 'Coherence', 'Topic_Diversity', 
                       'Coherence_norm', 'Topic_Diversity_norm', 'Combined_Score']])


## 6. Pareto Efficiency Analysis (Equal Weights)

Identify Pareto-efficient models - models where no other model is better in both metrics simultaneously.


In [ ]:
def identify_pareto(df, metrics):
    """
    Identify Pareto-efficient points for given metrics.
    
    Parameters:
    df (pd.DataFrame): DataFrame containing the data points
    metrics (list): List of column names to use for Pareto analysis
    
    Returns:
    np.array: Boolean array indicating whether each row is Pareto-efficient
    """
    pareto_efficient = np.ones(df.shape[0], dtype=bool)
    for i, row in df.iterrows():
        # If there are any other points that are strictly better, mark as not Pareto-efficient
        pareto_efficient[i] = not np.any(
            np.all(df[metrics].values >= row[metrics].values, axis=1) & 
            np.any(df[metrics].values > row[metrics].values, axis=1)
        )
    return pareto_efficient

# Identify Pareto-efficient models (overall)
df_clean['Pareto_Efficient_All'] = identify_pareto(
    df_clean, ['Coherence_norm', 'Topic_Diversity_norm']
)

# Identify Pareto-efficient models per embedding model
df_clean['Pareto_Efficient_PerModel'] = False
for model_name, group in df_clean.groupby('Embeddings_Model'):
    pareto_flags = identify_pareto(group, ['Coherence_norm', 'Topic_Diversity_norm'])
    df_clean.loc[group.index, 'Pareto_Efficient_PerModel'] = pareto_flags

pareto_all_count = df_clean['Pareto_Efficient_All'].sum()
pareto_per_model_count = df_clean['Pareto_Efficient_PerModel'].sum()

print(f"Pareto-efficient models (overall): {pareto_all_count}")
print(f"Pareto-efficient models (per model): {pareto_per_model_count}")


In [ ]:
# Create comprehensive scatter plot showing Pareto front
fig, ax = plt.subplots(figsize=(12, 8))

# Plot all models
sns.scatterplot(
    data=df_clean, 
    x='Topic_Diversity', 
    y='Coherence', 
    hue='Embeddings_Model',
    palette='Set2', 
    s=70, 
    alpha=0.7,
    ax=ax
)

# Highlight Pareto-efficient models
pareto_models = df_clean[df_clean['Pareto_Efficient_All']]
ax.scatter(
    pareto_models['Topic_Diversity'], 
    pareto_models['Coherence'],
    facecolors='none', 
    edgecolors='red', 
    s=200, 
    linewidths=2, 
    label='Pareto-efficient',
    zorder=10
)

ax.set_title('Pareto Front: Coherence vs. Topic Diversity (Equal Weights)', fontsize=14)
ax.set_xlabel('Topic Diversity', fontsize=12)
ax.set_ylabel('Coherence', fontsize=12)
ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(output_base / "figures/pareto_front_equal_weights.png", dpi=150, bbox_inches='tight')
plt.show()


In [ ]:
# Create per-model Pareto front plots
unique_models = df_clean['Embeddings_Model'].unique()
num_models = len(unique_models)
cols = 2
rows = math.ceil(num_models / cols)

fig, axes = plt.subplots(rows, cols, figsize=(15, 5 * rows))
axes = axes.flatten() if num_models > 1 else [axes]

for i, model_name in enumerate(unique_models):
    subset = df_clean[df_clean['Embeddings_Model'] == model_name]
    
    # Scatter plot of all runs
    axes[i].scatter(
        subset['Topic_Diversity'], 
        subset['Coherence'], 
        label='All Runs', 
        color='blue', 
        alpha=0.6,
        s=50
    )
    
    # Highlight Pareto-efficient runs
    pareto_subset = subset[subset['Pareto_Efficient_PerModel']]
    if len(pareto_subset) > 0:
        axes[i].scatter(
            pareto_subset['Topic_Diversity'], 
            pareto_subset['Coherence'], 
            label='Pareto Efficient', 
            color='red', 
            s=100,
            edgecolors='darkred',
            linewidths=1.5
        )
    
    axes[i].set_title(f'Pareto Front for {model_name}', fontsize=12)
    axes[i].set_xlabel('Topic Diversity', fontsize=10)
    axes[i].set_ylabel('Coherence', fontsize=10)
    axes[i].legend(fontsize=9)
    axes[i].grid(True, alpha=0.3)

# Hide unused subplots
for i in range(num_models, len(axes)):
    axes[i].axis('off')

plt.tight_layout()
plt.savefig(output_base / "figures/pareto_fronts_per_model.png", dpi=150, bbox_inches='tight')
plt.show()


In [ ]:
# Summary statistics by embedding model for Pareto-efficient models
pareto_efficient_models = df_clean[df_clean['Pareto_Efficient_All']]
pareto_grouped = pareto_efficient_models.groupby('Embeddings_Model').agg({
    'Coherence_norm': ['mean', 'std', 'count'],
    'Topic_Diversity_norm': ['mean', 'std', 'count'],
    'Combined_Score': ['mean', 'std', 'count']
}).reset_index()

pareto_grouped.columns = [
    'Embeddings_Model',
    'Coherence_mean', 'Coherence_std', 'Coherence_count',
    'Diversity_mean', 'Diversity_std', 'Diversity_count',
    'Combined_mean', 'Combined_std', 'Combined_count'
]

print("Performance of Embedding Models in Pareto-Efficient Models:")
display(pareto_grouped)

# Select top 10 Pareto-efficient models
top_10_equal_weights = pareto_efficient_models.nlargest(10, 'Combined_Score')
print(f"\nTop 10 Pareto-efficient Models (Equal Weights):")
display(top_10_equal_weights[['Embeddings_Model', 'Iteration', 'Coherence', 'Topic_Diversity',
                              'Coherence_norm', 'Topic_Diversity_norm', 'Combined_Score']])

# Save top 10 models
top_10_equal_weights.to_csv(output_base / "top_models/top_10_equal_weights.csv", index=False)
print(f"\nSaved top 10 models to: {output_base / 'top_models/top_10_equal_weights.csv'}")


## 7. Combined Score Calculation (Coherence Priority)

Recalculate combined score with higher weight on coherence (0.7) and lower weight on topic diversity (0.3).


In [ ]:
# Define weights for coherence priority strategy
weight_coherence = 0.7
weight_topic_diversity = 0.3

# Recalculate combined score
df_clean['Combined_Score'] = (
    weight_coherence * df_clean['Coherence_norm'] + 
    weight_topic_diversity * df_clean['Topic_Diversity_norm']
)

# Re-identify Pareto-efficient models
df_clean['Pareto_Efficient'] = identify_pareto(
    df_clean, ['Coherence_norm', 'Topic_Diversity_norm']
)

# Rank by combined score
df_sorted_priority = df_clean.sort_values(by='Combined_Score', ascending=False).reset_index(drop=True)

print(f"Top 10 Models Based on Combined Score (Coherence Priority):")
display(df_sorted_priority.head(10)[['Embeddings_Model', 'Iteration', 'Coherence', 'Topic_Diversity',
                                      'Coherence_norm', 'Topic_Diversity_norm', 'Combined_Score']])


## 8. Pareto Efficiency Analysis (Coherence Priority)


In [ ]:
# Filter Pareto-efficient models
pareto_efficient_priority = df_clean[df_clean['Pareto_Efficient']]
pareto_count_priority = len(pareto_efficient_priority)

print(f"Number of Pareto-efficient Models (Coherence Priority): {pareto_count_priority}")

# Create scatter plot for coherence-priority Pareto front
fig, ax = plt.subplots(figsize=(12, 8))

sns.scatterplot(
    data=df_clean, 
    x='Topic_Diversity', 
    y='Coherence', 
    hue='Embeddings_Model',
    palette='Set2', 
    s=70, 
    alpha=0.7,
    ax=ax
)

# Highlight Pareto-efficient models
pareto_models_priority = df_clean[df_clean['Pareto_Efficient']]
ax.scatter(
    pareto_models_priority['Topic_Diversity'], 
    pareto_models_priority['Coherence'],
    facecolors='none', 
    edgecolors='red', 
    s=200, 
    linewidths=2, 
    label='Pareto-efficient (Coherence Priority)',
    zorder=10
)

ax.set_title('Pareto Front: Coherence vs. Topic Diversity (Coherence Prioritized)', fontsize=14)
ax.set_xlabel('Topic Diversity', fontsize=12)
ax.set_ylabel('Coherence', fontsize=12)
ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(output_base / "figures/pareto_front_coherence_priority.png", dpi=150, bbox_inches='tight')
plt.show()

# Select top 10 Pareto-efficient models
top_10_coherence_priority = pareto_efficient_priority.nlargest(10, 'Combined_Score')
print(f"\nTop 10 Pareto-efficient Models (Coherence Priority):")
display(top_10_coherence_priority[['Embeddings_Model', 'Iteration', 'Coherence', 'Topic_Diversity',
                                    'Coherence_norm', 'Topic_Diversity_norm', 'Combined_Score']])

# Save top 10 models
top_10_coherence_priority.to_csv(output_base / "top_models/top_10_coherence_priority.csv", index=False)
print(f"\nSaved top 10 models to: {output_base / 'top_models/top_10_coherence_priority.csv'}")


## 9. Hyperparameter Analysis

Analyze relationships between hyperparameters and performance metrics for the top models.


In [ ]:
# Load the saved top 10 model sets
df_equal_weights = pd.read_csv(output_base / "top_models/top_10_equal_weights.csv")
df_coherence_priority = pd.read_csv(output_base / "top_models/top_10_coherence_priority.csv")

# Define hyperparameters
hyperparameters = [
    'bertopic__min_topic_size',
    'bertopic__top_n_words',
    'hdbscan__min_cluster_size',
    'hdbscan__min_samples',
    'umap__min_dist',
    'umap__n_neighbors',
    'vectorizer__min_df'
]

# Performance metrics
performance_metrics = ['Coherence_norm', 'Topic_Diversity_norm', 'Combined_Score']

print("Descriptive Statistics for Hyperparameters (Equal Weights):")
display(df_equal_weights[hyperparameters].describe())

print("\nDescriptive Statistics for Hyperparameters (Coherence Priority):")
display(df_coherence_priority[hyperparameters].describe())


In [ ]:
# Calculate correlation matrices
df_equal_filtered = df_equal_weights[hyperparameters + performance_metrics]
df_priority_filtered = df_coherence_priority[hyperparameters + performance_metrics]

correlation_equal = df_equal_filtered.corr()
correlation_priority = df_priority_filtered.corr()

print("Correlation Matrix (Equal Weights):")
display(correlation_equal)

print("\nCorrelation Matrix (Coherence Priority):")
display(correlation_priority)


In [ ]:
# Create normalized hyperparameter boxplots
scaler_hp = MinMaxScaler()
df_normalized_eq = df_equal_weights.copy()
df_normalized_pri = df_coherence_priority.copy()

df_normalized_eq[hyperparameters] = scaler_hp.fit_transform(df_equal_weights[hyperparameters])
df_normalized_pri[hyperparameters] = scaler_hp.fit_transform(df_coherence_priority[hyperparameters])

# Melt for plotting
df_melted_eq = df_normalized_eq.melt(
    value_vars=hyperparameters,
    var_name='Hyperparameter',
    value_name='Value'
)
df_melted_eq['Strategy'] = 'Equal Weights'

df_melted_pri = df_normalized_pri.melt(
    value_vars=hyperparameters,
    var_name='Hyperparameter',
    value_name='Value'
)
df_melted_pri['Strategy'] = 'Coherence Priority'

df_melted = pd.concat([df_melted_eq, df_melted_pri], ignore_index=True)

# Create boxplots
fig, ax = plt.subplots(figsize=(14, 6))
sns.boxplot(
    data=df_melted,
    x='Value',
    y='Hyperparameter',
    hue='Strategy',
    orient='h',
    ax=ax
)

ax.set_title('Boxplots of Normalized Hyperparameters', fontsize=16)
ax.set_xlabel('Normalized Values', fontsize=12)
ax.set_ylabel('Hyperparameter', fontsize=12)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(output_base / "figures/hyperparameter_boxplots.png", dpi=150, bbox_inches='tight')
plt.show()


In [ ]:
# Function to calculate Cohen's d
def cohen_d(group1, group2):
    """Calculate Cohen's d for two groups."""
    diff_mean = np.mean(group1) - np.mean(group2)
    pooled_std = np.sqrt(((np.std(group1, ddof=1) ** 2) + (np.std(group2, ddof=1) ** 2)) / 2)
    return diff_mean / pooled_std if pooled_std > 0 else 0

# Function to choose correlation test
def choose_test(df, parameter, metric):
    """Choose between Pearson or Spearman based on data distribution."""
    if abs(df[parameter].skew()) < 1:
        corr, p_value = pearsonr(df[parameter], df[metric])
        test_type = "Pearson"
    else:
        corr, p_value = spearmanr(df[parameter], df[metric])
        test_type = "Spearman"
    return corr, p_value, test_type

# Function to calculate Cohen's d between groups
def calculate_cohens_d(df, parameter, metric):
    """Split by median and calculate Cohen's d."""
    median_value = df[parameter].median()
    group1 = df[df[parameter] <= median_value][metric]
    group2 = df[df[parameter] > median_value][metric]
    return cohen_d(group1, group2)

# Analyze correlations and effect sizes
results_equal = {'Hyperparameter': [], 'Metric': [], 'Correlation': [], 'p-value': [], 'Test_Type': [], 'Cohen_d': []}
results_priority = {'Hyperparameter': [], 'Metric': [], 'Correlation': [], 'p-value': [], 'Test_Type': [], 'Cohen_d': []}

for param in hyperparameters:
    for metric in performance_metrics:
        # Equal weights
        corr, p_val, test_type = choose_test(df_equal_weights, param, metric)
        cohens_d_val = calculate_cohens_d(df_equal_weights, param, metric)
        results_equal['Hyperparameter'].append(param)
        results_equal['Metric'].append(metric)
        results_equal['Correlation'].append(corr)
        results_equal['p-value'].append(p_val)
        results_equal['Test_Type'].append(test_type)
        results_equal['Cohen_d'].append(cohens_d_val)
        
        # Coherence priority
        corr, p_val, test_type = choose_test(df_coherence_priority, param, metric)
        cohens_d_val = calculate_cohens_d(df_coherence_priority, param, metric)
        results_priority['Hyperparameter'].append(param)
        results_priority['Metric'].append(metric)
        results_priority['Correlation'].append(corr)
        results_priority['p-value'].append(p_val)
        results_priority['Test_Type'].append(test_type)
        results_priority['Cohen_d'].append(cohens_d_val)

# Convert to DataFrames
correlation_results_equal = pd.DataFrame(results_equal)
correlation_results_priority = pd.DataFrame(results_priority)

print("Correlation and Cohen's d Results (Equal Weights):")
display(correlation_results_equal)

print("\nCorrelation and Cohen's d Results (Coherence Priority):")
display(correlation_results_priority)

# Save results
correlation_results_equal.to_csv(output_base / "tables/correlation_analysis_equal_weights.csv", index=False)
correlation_results_priority.to_csv(output_base / "tables/correlation_analysis_coherence_priority.csv", index=False)

print(f"\nSaved correlation analysis to:")
print(f"  - {output_base / 'tables/correlation_analysis_equal_weights.csv'}")
print(f"  - {output_base / 'tables/correlation_analysis_coherence_priority.csv'}")
